--- Day 8: I Heard You Like Registers ---  
You receive a signal directly from the CPU.  
Because of your recent assistance with jump instructions,  
it would like you to compute the result of a series of unusual register instructions.  

Each instruction consists of several parts:  
the register to modify, whether to increase or decrease that register's value,  
the amount by which to increase or decrease it, and a condition.  
If the condition fails, skip the instruction without modifying the register.  
The registers all start at 0. The instructions look like this:  

b inc 5 if a > 1  
a inc 1 if b < 5  
c dec -10 if a >= 1  
c inc -20 if c == 10  
These instructions would be processed as follows:  

Because a starts at 0, it is not greater than 1, and so b is not modified.  
a is increased by 1 (to 1) because b is less than 5 (it is 0).  
c is decreased by -10 (to 10) because a is now greater than or equal to 1 (it is 1).  
c is increased by -20 (to -10) because c is equal to 10.  
After this process, the largest value in any register is 1.  

You might also encounter <= (less than or equal to) or != (not equal to).  
However, the CPU doesn't have the bandwidth to tell you what all the registers are named,  
and leaves that to you to determine.  

What is the largest value in any register after completing the instructions in your puzzle input?  

In [36]:
import numpy as np
import pandas as pd
import operator #https://docs.python.org/2/library/operator.html

In [37]:
def parse(string):
    """ Parse out name, inc/dec, condition"""
    s=string.split(' ') #['b', 'inc', '5', 'if', 'a', '>', '1']
    
    name=s[0]
    change=s[1]
    steps=int(s[2])
    conditions=[s[-3],s[-2],int(s[-1])]
    
    return name,change,steps,conditions

In [38]:
parse('b inc 5 if a > 1')

('b', 'inc', 5, ['a', '>', 1])

In [39]:
parse('bmm inc 977 if k <= -739')

('bmm', 'inc', 977, ['k', '<=', -739])

In [40]:
test=[
'b inc 5 if a > 1',
'a inc 1 if b < 5',
'c dec -10 if a >= 1',
'c inc -20 if c == 10'
]

In [41]:
test_parsed=[parse(t) for t in test]
test_parsed

[('b', 'inc', 5, ['a', '>', 1]),
 ('a', 'inc', 1, ['b', '<', 5]),
 ('c', 'dec', -10, ['a', '>=', 1]),
 ('c', 'inc', -20, ['c', '==', 10])]

In [42]:
# Create all names, start with value 0
main={} #empty dict
for t in test_parsed:
    main[t[0]]=0

In [43]:
main

{'a': 0, 'b': 0, 'c': 0}

In [44]:
main['a']+=1

In [45]:
main

{'a': 1, 'b': 0, 'c': 0}

In [46]:
def check_condition(array_of_conditions,dictionary):
    name=array_of_conditions[0]
    sign=array_of_conditions[1]
    back=array_of_conditions[2]
    
    front=dictionary[name]
    
    if sign=='>':
        return operator.gt(front,back)
    elif sign=='>=':
        return operator.ge(front,back)
    elif sign=='<':
        return operator.lt(front,back)
    elif sign=='<=':
        return operator.le(front,back)
    elif sign=='==':
        return operator.eq(front,back)
    elif sign=='!=':
        return operator.ne(front,back)

In [47]:
def compute(name,change,steps,conditions,dictionary):
    if check_condition(conditions,dictionary):
        if change=='inc':
            dictionary[name]+=steps
        elif change=='dec':
            dictionary[name]-=steps
        

In [48]:
for t in test_parsed:
    compute(t[0],t[1],t[2],t[3],main)

In [49]:
main

{'a': 2, 'b': 0, 'c': -10}

In [50]:
max(main.values())

2

In [51]:
with open('inputs.txt','r') as f:
    inputs=f.readlines()
    inputs=[i.replace('\n','') for i in inputs]
inputs[:5]

['v inc 523 if t == 6',
 'qen dec -450 if lht != 10',
 'jyg dec -378 if lb != -6',
 'k inc -994 if z < 6',
 'gjr inc -698 if hbq < 9']

In [52]:
inputs_parsed=[parse(i) for i in inputs]
inputs_parsed[:5]

[('v', 'inc', 523, ['t', '==', 6]),
 ('qen', 'dec', -450, ['lht', '!=', 10]),
 ('jyg', 'dec', -378, ['lb', '!=', -6]),
 ('k', 'inc', -994, ['z', '<', 6]),
 ('gjr', 'inc', -698, ['hbq', '<', 9])]

In [53]:
main={} #empty dict
for t in inputs_parsed:
    main[t[0]]=0

In [54]:
for t in inputs_parsed:
    compute(t[0],t[1],t[2],t[3],main)

In [55]:
main

{'bmm': -2801,
 'cj': 1328,
 'ee': -3860,
 'erl': -5233,
 'fg': -1234,
 'g': -542,
 'gjr': 328,
 'gx': -989,
 'hbq': 1525,
 'iwd': -2340,
 'jpc': -1859,
 'jyg': -1846,
 'k': -1240,
 'kfw': -6156,
 'lb': 1571,
 'lht': -4440,
 'ml': -5686,
 'mpj': -1,
 'osr': -3833,
 'pv': -952,
 'qen': 3612,
 't': -6133,
 'tu': 277,
 'u': -1685,
 'uz': -800,
 'v': 256,
 'z': -6365}

In [56]:
max(main.values())

3612

--- Part Two ---  
To be safe, the CPU also needs to know the highest value held in any register during this process  
so that it can decide how much memory to allocate to these operations.  
For example, in the above instructions, the highest value ever held was 10  
(in register c after the third instruction was evaluated).

In [63]:
def compute2(name,change,steps,conditions,dictionary):
    max_value=max(dictionary.values())
    if check_condition(conditions,dictionary):
        if change=='inc':
            dictionary[name]+=steps
            if max(dictionary.values())>max_value:
                max_value=max(dictionary.values())  
            
        elif change=='dec':
            dictionary[name]-=steps
            if max(dictionary.values())>max_value:
                max_value=max(dictionary.values())
    return max_value

In [64]:
#Test case
main={} #empty dict
for t in test_parsed:
    main[t[0]]=0

max_values=[]
for t in test_parsed:
    max_values.append(compute2(t[0],t[1],t[2],t[3],main))
    
max(max_values)

10

In [65]:
#Solution for part 2
main={} #empty dict
for t in inputs_parsed:
    main[t[0]]=0

max_values=[]
for t in inputs_parsed:
    max_values.append(compute2(t[0],t[1],t[2],t[3],main))
    
max(max_values)

3818